# หมายเหตุ
เป็นการทดลองเพื่อเปรียบเทียบประสิทธิภาพของ Embedding model

*   โดยจะใช้ โมเดล BGE - M3
*   จะเน้นค่า Recall@ k   เท่านั้น



คำเเนะนำ
  ให้ทำการเชื่อม google drive ก่อนทุกครั้ง


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


โหลดสิ่งที่จำเป็น

In [ ]:
!pip install -U langchain-community

In [ ]:
# ติดตั้ง PyTorch เวอร์ชันล่าสุด
!pip install torch==2.0.1

In [ ]:
# 🚀 ลบ PyTorch และ Transformers เก่าทั้งหมด
!pip uninstall -y torch torchvision torchaudio transformers

# 🚀 ติดตั้ง PyTorch ที่รองรับ CUDA ของ Colab (CUDA 11.8)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 🚀 ติดตั้ง Transformers เวอร์ชันที่รองรับ
!pip install transformers --upgrade

In [ ]:
!pip install -q langchain transformers chromadb requests
!pip install -q langchain chromadb requests

**English Evaluation dataset**

ทำการ embed คำถามของ Evaluation datasest ด้วย BGE-M3

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
import numpy as np

# 🚀 โหลดโมเดล BGE-M3
model_name = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 🚀 ฟังก์ชันทำ mean pooling สำหรับ embedding
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element: last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# 🚀 ฟังก์ชันสร้าง Embedding ของคำถามแบบ batch
def embed_text_batch(texts):
    encoded_input = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        model_output = model(**encoded_input)

    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return embeddings.cpu().numpy()

# 🚀 โหลด Evaluation Dataset
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_datasetv2.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 สร้าง Embeddings สำหรับคำถามทั้งหมด (Batch)
batch_size = 10
all_embeddings = []

for i in range(0, len(evaluation_data), batch_size):
    batch_questions = [item['question'] for item in evaluation_data[i:i + batch_size]]
    batch_embeddings = embed_text_batch(batch_questions)
    all_embeddings.extend(batch_embeddings.tolist())

    print(f"✅ batch ที่ {i // batch_size + 1} embed แล้ว")

    if device.type == "cuda":
        torch.cuda.empty_cache()

# 🚀 เพิ่ม Embeddings กลับเข้า Evaluation Dataset
for i, item in enumerate(evaluation_data):
    item['embedding'] = all_embeddings[i]

print("✅ สร้าง Embeddings สำหรับคำถามใน Evaluation Dataset สำเร็จ")

# 🚀 กำหนดเส้นทางที่ต้องการบันทึกไฟล์
output_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2_BGE_M3.json'

# 🚀 บันทึก Evaluation Dataset พร้อม Embeddings ลงไฟล์ JSON
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(evaluation_data, f, ensure_ascii=False, indent=4)

print(f"✅ บันทึก Evaluation Dataset พร้อม Embeddings ที่: {output_file}")


เชื่อม vector database

In [ ]:
# 🚀 2. เชื่อมต่อกับ Chroma Database v5 ที่จะสร้างใหม่
import chromadb

db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v5'  # ✅ เปลี่ยน path เป็น v5
client = chromadb.PersistentClient(path=db_path)

# 🚀 โหลด Collection ที่มีอยู่แล้ว (หรือจะสร้างถ้ายังไม่มี)
collection_name = "recommendations"
collection = client.get_collection(name=collection_name)

print("✅ เชื่อมต่อกับ Chroma DB v5 และโหลดคอลเลกชัน 'recommendations' สำเร็จ")


เช็คว่าสามารถดึงข้อมูลได้มั้ย

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings จาก BGE-M3
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2_BGE_M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset (BGE-M3) จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB เวอร์ชัน v5
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v5'  # ✅ อัปเดตเป็น v5
client = chromadb.PersistentClient(path=db_path)

# 🚀 โหลด Collection "recommendations"
collection = client.get_collection(name="recommendations")
print("✅ เชื่อมต่อกับ Chroma DB v5 และโหลดคอลเลกชัน 'recommendations' สำเร็จ")

# 🚀 ฟังก์ชัน RAG ดึงคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(question_embedding, top_k=3):
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=top_k
    )
    if results and 'documents' in results and len(results['documents']) > 0:
        return results['documents'][0]
    return []

# 🚀 ทดลองดึงคำแนะนำที่ใกล้เคียงสำหรับคำถามแรก
test_question = evaluation_data[0]['question']
test_embedding = evaluation_data[0]['embedding']

recommendations = retrieve_recommendations(test_embedding)
print(f"\n🔎 คำถาม: {test_question}\n✅ คำแนะนำที่ใกล้เคียง:\n")

for idx, rec in enumerate(recommendations, 1):
    print(f"{idx}. {rec}")


ให้ค่า K = 3

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings (จาก BGE-M3)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2_BGE_M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset (BGE-M3) จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB เวอร์ชัน v5
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v5'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG (Retrieve) เพื่อค้นหาคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(query_embedding, top_k=3):
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0] if results['documents'] else []

# 🚀 ใช้ RAG ค้นหาคำแนะนำสำหรับทุกคำถามใน Dataset
rag_results = []

print("🔄 กำลังดึงคำแนะนำที่ใกล้เคียงด้วย RAG...")

for idx, item in enumerate(evaluation_data, 1):
    query_embedding = item['embedding']
    recommendations = retrieve_recommendations(query_embedding, top_k=3)

    rag_results.append({
        "question": item['question'],
        "answer": item['answer'],
        "category": item.get('category', 'Unknown'),
        "recommendations": recommendations
    })

    if idx % 100 == 0 or idx == len(evaluation_data):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(evaluation_data)} คำถาม")

# 🚀 บันทึกผลลัพธ์ RAG
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v2_BGE_M3_K=3.json'
with open(rag_results_file, 'w', encoding='utf-8') as f:
    json.dump(rag_results, f, ensure_ascii=False, indent=4)

print(f'✅ บันทึกผลลัพธ์ RAG ที่: {rag_results_file}')


หาค่า Recall

In [2]:
import json

# 🚀 โหลด Evaluation Dataset (มี embeddings)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2_BGE_M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# 🚀 โหลด RAG Results (จาก BGE-M3)
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v2_BGE_M3_K=3.json'
with open(rag_results_file, 'r', encoding='utf-8') as f:
    rag_results = json.load(f)

K = 3  # จำนวนคำแนะนำที่ดึงมา

# 🚀 ฟังก์ชันคำนวณ Precision@K
def precision_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r.lower().strip() in relevant)
    return hits / k if k > 0 else 0

# 🚀 ฟังก์ชันคำนวณ Recall@K
def recall_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r.lower().strip() in relevant)
    return hits / len(relevant) if relevant else 0

# 🚀 ฟังก์ชันคำนวณ Mean Reciprocal Rank (MRR)
def mean_reciprocal_rank(relevant, retrieved):
    for idx, rec in enumerate(retrieved, 1):
        if rec.lower().strip() in relevant:
            return 1 / idx
    return 0

# 🚀 เก็บผลลัพธ์
precisions = []
recalls = []
mrrs = []

print("🔄 กำลังคำนวณ Precision, Recall, และ MRR...")

for idx, item in enumerate(rag_results, 1):
    if 'answer' not in item or 'recommendations' not in item:
        print(f"⚠️ ข้อที่ {idx} ไม่มีคำตอบหรือคำแนะนำ ข้ามไป...")
        continue

    # ✅ รองรับคำตอบหลายรายการ
    if isinstance(item['answer'], list):
        relevant_answers = [ans.lower().strip() for ans in item['answer']]
    else:
        relevant_answers = [item['answer'].lower().strip()]

    retrieved = [r.lower().strip() for r in item['recommendations']]

    p = precision_at_k(relevant_answers, retrieved, K)
    r = recall_at_k(relevant_answers, retrieved, K)
    mrr = mean_reciprocal_rank(relevant_answers, retrieved)

    precisions.append(p)
    recalls.append(r)
    mrrs.append(mrr)

    if idx % 100 == 0 or idx == len(rag_results):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(rag_results)} คำถาม")

# 🚀 สรุปผลลัพธ์
avg_precision = sum(precisions) / len(precisions) if precisions else 0
avg_recall = sum(recalls) / len(recalls) if recalls else 0
avg_mrr = sum(mrrs) / len(mrrs) if mrrs else 0

print("\n🔎 ผลลัพธ์การประเมินผลด้วย RAG (BGE-M3)")
print(f"Precision@{K}: {avg_precision:.3f}")
print(f"Recall@{K}: {avg_recall:.3f}")
print(f"Mean Reciprocal Rank (MRR): {avg_mrr:.3f}")


🔄 กำลังคำนวณ Precision, Recall, และ MRR...
✅ ประมวลผลแล้ว 100/1370 คำถาม
✅ ประมวลผลแล้ว 200/1370 คำถาม
✅ ประมวลผลแล้ว 300/1370 คำถาม
✅ ประมวลผลแล้ว 400/1370 คำถาม
✅ ประมวลผลแล้ว 500/1370 คำถาม
✅ ประมวลผลแล้ว 600/1370 คำถาม
✅ ประมวลผลแล้ว 700/1370 คำถาม
✅ ประมวลผลแล้ว 800/1370 คำถาม
✅ ประมวลผลแล้ว 900/1370 คำถาม
✅ ประมวลผลแล้ว 1000/1370 คำถาม
✅ ประมวลผลแล้ว 1100/1370 คำถาม
✅ ประมวลผลแล้ว 1200/1370 คำถาม
✅ ประมวลผลแล้ว 1300/1370 คำถาม
✅ ประมวลผลแล้ว 1370/1370 คำถาม

🔎 ผลลัพธ์การประเมินผลด้วย RAG (BGE-M3)
Precision@3: 0.180
Recall@3: 0.539
Mean Reciprocal Rank (MRR): 0.453


ให้ค่า K = 5

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings (จาก BGE-M3)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2_BGE_M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset (BGE-M3) จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB เวอร์ชัน v5
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v5'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG (Retrieve) เพื่อค้นหาคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(query_embedding, top_k=5):
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0] if results['documents'] else []

# 🚀 ใช้ RAG ค้นหาคำแนะนำสำหรับทุกคำถามใน Dataset
rag_results = []

print("🔄 กำลังดึงคำแนะนำที่ใกล้เคียงด้วย RAG...")

for idx, item in enumerate(evaluation_data, 1):
    query_embedding = item['embedding']
    recommendations = retrieve_recommendations(query_embedding, top_k=5)

    rag_results.append({
        "question": item['question'],
        "answer": item['answer'],
        "category": item.get('category', 'Unknown'),
        "recommendations": recommendations
    })

    if idx % 100 == 0 or idx == len(evaluation_data):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(evaluation_data)} คำถาม")

# 🚀 บันทึกผลลัพธ์ RAG
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v2_BGE_M3_K=5.json'
with open(rag_results_file, 'w', encoding='utf-8') as f:
    json.dump(rag_results, f, ensure_ascii=False, indent=4)

print(f'✅ บันทึกผลลัพธ์ RAG ที่: {rag_results_file}')


หาค่า Recall

In [3]:
import json

# 🚀 โหลด Evaluation Dataset (มี embeddings)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2_BGE_M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# 🚀 โหลด RAG Results (จาก BGE-M3)
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v2_BGE_M3_K=5.json'
with open(rag_results_file, 'r', encoding='utf-8') as f:
    rag_results = json.load(f)

K = 5  # จำนวนคำแนะนำที่ดึงมา

# 🚀 ฟังก์ชันคำนวณ Precision@K
def precision_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r.lower().strip() in relevant)
    return hits / k if k > 0 else 0

# 🚀 ฟังก์ชันคำนวณ Recall@K
def recall_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r.lower().strip() in relevant)
    return hits / len(relevant) if relevant else 0

# 🚀 ฟังก์ชันคำนวณ Mean Reciprocal Rank (MRR)
def mean_reciprocal_rank(relevant, retrieved):
    for idx, rec in enumerate(retrieved, 1):
        if rec.lower().strip() in relevant:
            return 1 / idx
    return 0

# 🚀 เก็บผลลัพธ์
precisions = []
recalls = []
mrrs = []

print("🔄 กำลังคำนวณ Precision, Recall, และ MRR...")

for idx, item in enumerate(rag_results, 1):
    if 'answer' not in item or 'recommendations' not in item:
        print(f"⚠️ ข้อที่ {idx} ไม่มีคำตอบหรือคำแนะนำ ข้ามไป...")
        continue

    # ✅ รองรับคำตอบหลายรายการ
    if isinstance(item['answer'], list):
        relevant_answers = [ans.lower().strip() for ans in item['answer']]
    else:
        relevant_answers = [item['answer'].lower().strip()]

    retrieved = [r.lower().strip() for r in item['recommendations']]

    p = precision_at_k(relevant_answers, retrieved, K)
    r = recall_at_k(relevant_answers, retrieved, K)
    mrr = mean_reciprocal_rank(relevant_answers, retrieved)

    precisions.append(p)
    recalls.append(r)
    mrrs.append(mrr)

    if idx % 100 == 0 or idx == len(rag_results):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(rag_results)} คำถาม")

# 🚀 สรุปผลลัพธ์
avg_precision = sum(precisions) / len(precisions) if precisions else 0
avg_recall = sum(recalls) / len(recalls) if recalls else 0
avg_mrr = sum(mrrs) / len(mrrs) if mrrs else 0

print("\n🔎 ผลลัพธ์การประเมินผลด้วย RAG (BGE-M3)")
print(f"Precision@{K}: {avg_precision:.3f}")
print(f"Recall@{K}: {avg_recall:.3f}")
print(f"Mean Reciprocal Rank (MRR): {avg_mrr:.3f}")


🔄 กำลังคำนวณ Precision, Recall, และ MRR...
✅ ประมวลผลแล้ว 100/1370 คำถาม
✅ ประมวลผลแล้ว 200/1370 คำถาม
✅ ประมวลผลแล้ว 300/1370 คำถาม
✅ ประมวลผลแล้ว 400/1370 คำถาม
✅ ประมวลผลแล้ว 500/1370 คำถาม
✅ ประมวลผลแล้ว 600/1370 คำถาม
✅ ประมวลผลแล้ว 700/1370 คำถาม
✅ ประมวลผลแล้ว 800/1370 คำถาม
✅ ประมวลผลแล้ว 900/1370 คำถาม
✅ ประมวลผลแล้ว 1000/1370 คำถาม
✅ ประมวลผลแล้ว 1100/1370 คำถาม
✅ ประมวลผลแล้ว 1200/1370 คำถาม
✅ ประมวลผลแล้ว 1300/1370 คำถาม
✅ ประมวลผลแล้ว 1370/1370 คำถาม

🔎 ผลลัพธ์การประเมินผลด้วย RAG (BGE-M3)
Precision@5: 0.119
Recall@5: 0.596
Mean Reciprocal Rank (MRR): 0.467


ให้ค่า K = 10

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings (จาก BGE-M3)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2_BGE_M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset (BGE-M3) จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB เวอร์ชัน v5
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v5'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG (Retrieve) เพื่อค้นหาคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(query_embedding, top_k=10):
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0] if results['documents'] else []

# 🚀 ใช้ RAG ค้นหาคำแนะนำสำหรับทุกคำถามใน Dataset
rag_results = []

print("🔄 กำลังดึงคำแนะนำที่ใกล้เคียงด้วย RAG...")

for idx, item in enumerate(evaluation_data, 1):
    query_embedding = item['embedding']
    recommendations = retrieve_recommendations(query_embedding, top_k=10)

    rag_results.append({
        "question": item['question'],
        "answer": item['answer'],
        "category": item.get('category', 'Unknown'),
        "recommendations": recommendations
    })

    if idx % 100 == 0 or idx == len(evaluation_data):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(evaluation_data)} คำถาม")

# 🚀 บันทึกผลลัพธ์ RAG
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v2_BGE_M3_K=10.json'
with open(rag_results_file, 'w', encoding='utf-8') as f:
    json.dump(rag_results, f, ensure_ascii=False, indent=4)

print(f'✅ บันทึกผลลัพธ์ RAG ที่: {rag_results_file}')


หาค่า Recall

In [4]:
import json

# 🚀 โหลด Evaluation Dataset (มี embeddings)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2_BGE_M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# 🚀 โหลด RAG Results (จาก BGE-M3)
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v2_BGE_M3_K=10.json'
with open(rag_results_file, 'r', encoding='utf-8') as f:
    rag_results = json.load(f)

K = 10  # จำนวนคำแนะนำที่ดึงมา

# 🚀 ฟังก์ชันคำนวณ Precision@K
def precision_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r.lower().strip() in relevant)
    return hits / k if k > 0 else 0

# 🚀 ฟังก์ชันคำนวณ Recall@K
def recall_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r.lower().strip() in relevant)
    return hits / len(relevant) if relevant else 0

# 🚀 ฟังก์ชันคำนวณ Mean Reciprocal Rank (MRR)
def mean_reciprocal_rank(relevant, retrieved):
    for idx, rec in enumerate(retrieved, 1):
        if rec.lower().strip() in relevant:
            return 1 / idx
    return 0

# 🚀 เก็บผลลัพธ์
precisions = []
recalls = []
mrrs = []

print("🔄 กำลังคำนวณ Precision, Recall, และ MRR...")

for idx, item in enumerate(rag_results, 1):
    if 'answer' not in item or 'recommendations' not in item:
        print(f"⚠️ ข้อที่ {idx} ไม่มีคำตอบหรือคำแนะนำ ข้ามไป...")
        continue

    # ✅ รองรับคำตอบหลายรายการ
    if isinstance(item['answer'], list):
        relevant_answers = [ans.lower().strip() for ans in item['answer']]
    else:
        relevant_answers = [item['answer'].lower().strip()]

    retrieved = [r.lower().strip() for r in item['recommendations']]

    p = precision_at_k(relevant_answers, retrieved, K)
    r = recall_at_k(relevant_answers, retrieved, K)
    mrr = mean_reciprocal_rank(relevant_answers, retrieved)

    precisions.append(p)
    recalls.append(r)
    mrrs.append(mrr)

    if idx % 100 == 0 or idx == len(rag_results):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(rag_results)} คำถาม")

# 🚀 สรุปผลลัพธ์
avg_precision = sum(precisions) / len(precisions) if precisions else 0
avg_recall = sum(recalls) / len(recalls) if recalls else 0
avg_mrr = sum(mrrs) / len(mrrs) if mrrs else 0

print("\n🔎 ผลลัพธ์การประเมินผลด้วย RAG (BGE-M3)")
print(f"Precision@{K}: {avg_precision:.3f}")
print(f"Recall@{K}: {avg_recall:.3f}")
print(f"Mean Reciprocal Rank (MRR): {avg_mrr:.3f}")


🔄 กำลังคำนวณ Precision, Recall, และ MRR...
✅ ประมวลผลแล้ว 100/1370 คำถาม
✅ ประมวลผลแล้ว 200/1370 คำถาม
✅ ประมวลผลแล้ว 300/1370 คำถาม
✅ ประมวลผลแล้ว 400/1370 คำถาม
✅ ประมวลผลแล้ว 500/1370 คำถาม
✅ ประมวลผลแล้ว 600/1370 คำถาม
✅ ประมวลผลแล้ว 700/1370 คำถาม
✅ ประมวลผลแล้ว 800/1370 คำถาม
✅ ประมวลผลแล้ว 900/1370 คำถาม
✅ ประมวลผลแล้ว 1000/1370 คำถาม
✅ ประมวลผลแล้ว 1100/1370 คำถาม
✅ ประมวลผลแล้ว 1200/1370 คำถาม
✅ ประมวลผลแล้ว 1300/1370 คำถาม
✅ ประมวลผลแล้ว 1370/1370 คำถาม

🔎 ผลลัพธ์การประเมินผลด้วย RAG (BGE-M3)
Precision@10: 0.067
Recall@10: 0.674
Mean Reciprocal Rank (MRR): 0.477





**Thai Evaluation dataset**

ทำการ Embedding ด้วยโมเดล BGE - M3

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import json

# 🔧 กำหนดเส้นทางไฟล์ใหม่สำหรับ BGE-M3
input_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_TH.json'
output_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_TH_embedded_BGE_M3.json'

# 🚀 โหลดโมเดล BGE-M3 จาก Hugging Face
model_name = 'BAAI/bge-m3'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

# ตรวจสอบว่ามี GPU ใช้งานไหม แล้วย้ายโมเดลไปที่ device นั้น
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 🔥 ฟังก์ชันสร้าง Embedding ด้วย mean pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # ใช้ last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def create_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        model_output = model(**inputs)
    embedding = mean_pooling(model_output, inputs['attention_mask'])
    return embedding[0].cpu().numpy().tolist()

# 🚀 โหลดข้อมูลจากไฟล์
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"✅ รวมคำถามทั้งหมด: {len(data)}")

# เริ่มสร้าง embedding สำหรับแต่ละคำถาม
for index, item in enumerate(data, start=1):
    text = item.get("translated_question", "")
    if text:
        item['question_embedding'] = create_embedding(text)
        print(f"✅ คำถามที่ {index} ได้รับการสร้าง Embedding แล้ว")

# 💾 บันทึกไฟล์ผลลัพธ์
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("✅ เสร็จสิ้นการสร้าง Embedding ทั้งหมดด้วย BGE-M3")


test ว่าเชื่อมกับ Evaluation dataset ได้มั้ย

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings (แก้ชื่อไฟล์ให้ตรงกับไฟล์ที่สร้าง embedding ล่าสุด)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_TH_embedded_BGE_M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset พร้อม Embeddings จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB (แก้ path ให้ถูกต้องตามฐานข้อมูลใหม่)
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v5'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG ดึงคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(question_embedding, top_k=3):
    # ตรวจสอบชนิดของ embedding ให้เป็น list (ถ้าเป็น numpy array หรือ torch tensor ให้แปลงก่อน)
    if not isinstance(question_embedding, list):
        question_embedding = question_embedding.tolist()

    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=top_k
    )
    return results['documents'][0] if results['documents'] else []

print("✅ พร้อมสำหรับการค้นหาคำแนะนำด้วย RAG")

# 🚀 ทดลองดึงคำแนะนำที่ใกล้เคียงสำหรับคำถามแรก
test_question = evaluation_data[0].get('translated_question', evaluation_data[0].get('question', ''))
test_embedding = evaluation_data[0]['question_embedding']

recommendations = retrieve_recommendations(test_embedding)
print(f"\n🔎 คำถาม: {test_question}\n✅ คำแนะนำที่ใกล้เคียง:\n")

for idx, rec in enumerate(recommendations, 1):
    print(f"{idx}. {rec}")


ให้ค่า K = 3

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_TH_embedded_BGE_M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset พร้อม Embeddings จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v5'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG (Retrieve) เพื่อค้นหาคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(query_embedding, top_k=3):
    # ถ้า embedding ไม่ใช่ list ให้แปลงเป็น list ก่อน
    if not isinstance(query_embedding, list):
        query_embedding = query_embedding.tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0] if results['documents'] else []

# 🚀 ใช้ RAG ค้นหาคำแนะนำสำหรับทุกคำถามใน Dataset
rag_results = []

print("🔄 กำลังดึงคำแนะนำที่ใกล้เคียงด้วย RAG...")

for idx, item in enumerate(evaluation_data, 1):
    if 'question_embedding' not in item:
        print(f"⚠️ ข้อที่ {idx} ไม่มี key 'question_embedding' ข้ามไป...")
        continue

    query_embedding = item['question_embedding']
    recommendations = retrieve_recommendations(query_embedding, top_k=3)

    rag_results.append({
        "question": item.get('translated_question', ''),
        "answer": item.get('translated_answer', ''),
        "recommendations": recommendations
    })

    if idx % 100 == 0 or idx == len(evaluation_data):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(evaluation_data)} คำถาม")

# 🚀 บันทึกผลลัพธ์ RAG
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v2_TH_BGE_M3_K=3.json'
with open(rag_results_file, 'w', encoding='utf-8') as f:
    json.dump(rag_results, f, ensure_ascii=False, indent=4)

print(f'✅ บันทึกผลลัพธ์ RAG ที่: {rag_results_file}')


หาค่า Recall

In [ ]:
import json

# 🚀 โหลดไฟล์ evaluation dataset (คำถาม-คำตอบจริง)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_TH_embedded_BGE_M3.json'  # แก้เป็นไฟล์ล่าสุด
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# 🚀 โหลดไฟล์ผลลัพธ์ RAG (คำถาม + คำแนะนำที่ดึงมา)
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v2_TH_BGE_M3_K=3.json'  # แก้ชื่อไฟล์ตามที่บันทึกไว้ก่อนหน้า
with open(rag_results_file, 'r', encoding='utf-8') as f:
    rag_results = json.load(f)

K = 3  # จำนวนคำแนะนำที่ดึงมา (top_k)

# 🚀 ฟังก์ชันคำนวณ Precision@K
def precision_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / k if k > 0 else 0

# 🚀 ฟังก์ชันคำนวณ Recall@K
def recall_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / len(relevant) if relevant else 0

# 🚀 ฟังก์ชันคำนวณ Mean Reciprocal Rank (MRR)
def mean_reciprocal_rank(relevant, retrieved):
    for idx, rec in enumerate(retrieved, 1):
        if rec in relevant:
            return 1 / idx
    return 0

# 🚀 เก็บผลลัพธ์คำนวณต่างๆ
precisions = []
recalls = []
mrrs = []

print("🔄 กำลังคำนวณ Precision, Recall, และ MRR...")

for idx, item in enumerate(rag_results, 1):
    # ตรวจสอบว่ามีคำตอบและคำแนะนำหรือไม่
    if 'translated_answer' not in item or 'recommendations' not in item:
        print(f"⚠️ ข้อที่ {idx} ไม่มีคำตอบหรือคำแนะนำ ข้ามไป...")
        continue

    # คำตอบจริง (รองรับทั้ง str หรือ list)
    relevant_answers = [item['translated_answer']] if isinstance(item['translated_answer'], str) else item['translated_answer']

    # คำแนะนำที่ดึงมา (list of strings)
    retrieved = item['recommendations']

    # คำนวณ Metrics
    p = precision_at_k(relevant_answers, retrieved, K)
    r = recall_at_k(relevant_answers, retrieved, K)
    mrr = mean_reciprocal_rank(relevant_answers, retrieved)

    precisions.append(p)
    recalls.append(r)
    mrrs.append(mrr)

    if idx % 100 == 0 or idx == len(rag_results):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(rag_results)} คำถาม")

# 🚀 คำนวณค่าเฉลี่ยของ Precision, Recall, MRR ทั้งหมด
avg_precision = sum(precisions) / len(precisions) if precisions else 0
avg_recall = sum(recalls) / len(recalls) if recalls else 0
avg_mrr = sum(mrrs) / len(mrrs) if mrrs else 0

print("\n🔎 ผลลัพธ์การประเมินผลด้วย RAG")
print(f"Precision@{K}: {avg_precision:.3f}")
print(f"Recall@{K}: {avg_recall:.3f}")
print(f"Mean Reciprocal Rank (MRR): {avg_mrr:.3f}")


🔄 กำลังคำนวณ Precision, Recall, และ MRR...
✅ ประมวลผลแล้ว 100/1370 คำถาม
✅ ประมวลผลแล้ว 200/1370 คำถาม
✅ ประมวลผลแล้ว 300/1370 คำถาม
✅ ประมวลผลแล้ว 400/1370 คำถาม
✅ ประมวลผลแล้ว 500/1370 คำถาม
✅ ประมวลผลแล้ว 600/1370 คำถาม
✅ ประมวลผลแล้ว 700/1370 คำถาม
✅ ประมวลผลแล้ว 800/1370 คำถาม
✅ ประมวลผลแล้ว 900/1370 คำถาม
✅ ประมวลผลแล้ว 1000/1370 คำถาม
✅ ประมวลผลแล้ว 1100/1370 คำถาม
✅ ประมวลผลแล้ว 1200/1370 คำถาม
✅ ประมวลผลแล้ว 1300/1370 คำถาม
✅ ประมวลผลแล้ว 1370/1370 คำถาม

🔎 ผลลัพธ์การประเมินผลด้วย RAG
Precision@3: 0.169
Recall@3: 0.506
Mean Reciprocal Rank (MRR): 0.431
